# Harware Control

### Microscopes
Base it off of [micromanager](https://micro-manager.org/)

Python interfaces:

- [pycromanger](https://pycro-manager.readthedocs.io/en/latest/)
- [pymmcore](https://github.com/micro-manager/pymmcore#pymmcore-python-bindings-for-mmcore)


Comparison https://github.com/micro-manager/pycro-manager/issues/132


Pycromanager has some limitations and saves data into a weird file format that doesn't work with `tifffile` (see this github [issue](https://github.com/micro-manager/NDTiffStorage/issues/12)). However it has documentation which is a significant plus.

`pymmcore` is much more performant than `pycromanager` and offers far more detailed control of the scope. However, the documentation is essentially non-existant:

> We do not currently have Python-specific documentation for CMMCore. The C++ documentation is the best resource.



Despite those limitations both options allow for full featured control of microscope from python which opens the door to lots of exciting options:

1. Live analysis scripts that adapt the acquistion you are running
    - Could follow a cell in XY 
    - Use world class Neural Nets to do software autofocus
2. Easiest way to integrate external instrumentation into a multidimensional acquisition


Example:

Ian combined a traditional `BF` + `GFP` MDA with acuiring Raman spectra of single cells with the laser aiming system built through Python. This is a great example of one of Python's strengths of acting as `glue` between languages and programs.

The required the controls for each of:
- `Microscope`
- `Galvo mirror`
- `Laser Shutter`
- `Spectrometer`


### General Instrumentation

There is no general solution here - every instrument will have it's own idiosyncracies and you will absolutely need to read the documentation provided by the manufacturer.


For better or worse Serial communication is still the lingua franca of many scientific instruments.

Basic approach:

1. Hope that the manufacturer has published documentation
2. Read that documentation
3. Make wrapper an convenience classes

Helpful libraries:
- [pyserial](https://pyserial.readthedocs.io/en/latest/)
- [pyvisa](https://pyvisa.readthedocs.io/en/latest/)




Example:

I had a `Rigol DS1000E` oscilloscope that I wanted to integrate in a python control script.

So I read the documentation: https://www.batronix.com/pdf/Rigol/ProgrammingGuide/DS1000DE_ProgrammingGuide_EN.pdf
and then sat at the computer for hours being confused and trying things until I figured everything out.

```python
def read_scope(self, MODE="RAW"):
    scope = self.scope # scope is a pyvisa object

    scope.write(":STOP")
    scope.write(":WAV:POIN:MODE {:}".format(MODE))

    scope.write(":TRIG:EDGE:SWEep SINGle")  # Single shot trigger
    scope.write(":TRIG:EDGE:LEVel 3")  # range is -6*Scale  | +6*Scale, scale in v/div

    timescale = np.float(scope.query_ascii_values(":TIM:SCAL?")[0])
    timeoffset = np.float(scope.query_ascii_values(":TIM:OFFS?")[0])

    voltscale = np.array([0, 0], dtype=np.float)
    voltscale[0] = np.float(scope.query_ascii_values(":CHAN1:SCAL?")[0])
    voltscale[1] = np.float(scope.query_ascii_values(":CHAN2:SCAL?")[0])

    voltoffset = np.array([0, 0], dtype=np.float)
    voltoffset[0] = np.float(scope.query_ascii_values(":CHAN1:OFFS?")[0])
    voltoffset[1] = np.float(scope.query_ascii_values(":CHAN2:OFFS?")[0])

    npts = 8192 if MODE == "RAW" else 600
    t = np.arange(-int(npts / 2), int(npts / 2)) * timescale / 50 + timeoffset
    volts = np.zeros([2, npts])
    scope.write(":WAV:DATA? CHAN1")
    data1 = np.frombuffer(scope.read_raw()[10:], "B")
    scope.write(":WAV:DATA? CHAN2")
    data2 = np.frombuffer(scope.read_raw()[10:], "B")
    scope.write(":STOP")
    volts[0] = self.scale_data(data1, voltscale[0], voltoffset[0])
    volts[1] = self.scale_data(data2, voltscale[1], voltoffset[1])
    return t, volts
```

#### Arduino

General purpose microcontroller. Can't speak highly enough of their versatility.

Can send serial commands using `pyserial` and then have an easy interface to the physical world.

Helpful things:
- [detect-arduino](https://github.com/ianhi/detect-arduino) (automatically determine which serial port the arduino is on)
- Can write custom arduino code and serial handling logic
- Or can use a protcol like `pyfirmata` https://realpython.com/arduino-python/